In [1]:
#@title
### EP2 MAC0417/MAC5768
##################################################################
# AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMEROUSP,  #
# DECLARO QUE SOU O ÚNICO AUTORE RESPONSÁVEL PELA RESOLUÇÃO      #
# DESTE EP.                                                      #
# TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM,   #
# SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM          #
# DESONESTIDADE ACADÊMICA OU PLÁGIO.                             #
#                                                                #
# DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS        #
# DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A          #
# SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E        #
# DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS    #
# DEFINIDOS NO CÓDIGO DE ÉTICA DA USP.                           #
#                                                                # 
# ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO         #
# CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR             #
# DESONESTIDADE ACADÊMICA.                                       #
#                                                                #
#                                                                #
# Nome: Luiz Gustavo Pina de Sales                               #
# NUSP: 10736991                                                 #
# Turma: 2024145                                                 #
# Prof.: Ronaldo Fumio Hashimoto                                 #
#################################################################

# EP2 - Data Augmentation
Neste notebook temos as aplicações de funções de data augmentation sobre as imagens coletadas para análise no EP1.

Com base no feedback do EP1, foram corrigidos os seguintes pontos:
 - A estrutura de pastas para cada classe foi removida e agora todas as imagens compartilham o mesmo diretório respectivo de seu grupo (original, originalGrayDataset e augmentedDataset)
 - As duplicações das imagens também foi removida, agora temos as imagens únicas de acordo com suas características.   

## Importações
Realiza as importações das bibliotecas necessárias para execução do notebook, de acordo com as permitidas para uso pelo enunciado.

In [4]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import cv2
from cv2 import resize, imwrite
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas
import os 

## Código principal para aquisição das imagens

No código abaixo temos um banco de imagens que irá conter um dicionário com a seguinte hierarquia de chaves:

    Classe -> Tempo -> Local -> Fundo -> Imagens

Todas as outras funções irão acessar as imagens através deste dicionário.
Este código foi adaptado do EP1 para permitir o uso com a nova estrutura do banco de imagens.

In [7]:
bancoImagens = {}
quantidadeClasses = 0
quantidadeImagens = 0
resolucaoImagens = ""

def loadImages(path, quantidadeClasses, quantidadeImagens, resolucaoImagens, bancoImagens):
    imagens = os.listdir(path)
    
    for imagem in imagens:
        metadata = imagem.split(",")
        if metadata[0] not in bancoImagens.keys():
            bancoImagens[metadata[0]] = {}
            
        currentLevel = bancoImagens[metadata[0]]
        for metadataIndex in range(1,len(metadata)-1):
            if(metadata[metadataIndex] not in currentLevel): 
                currentLevel[metadata[metadataIndex]] = {}
                currentLevel = currentLevel[metadata[metadataIndex]]
            else: currentLevel = currentLevel[metadata[metadataIndex]] 

        local = metadata[-1].strip().replace(".jpg", "")
        currentLevel[local] = imread(path+imagem)
        quantidadeImagens += 1
        resolucaoImagens = f"{len(currentLevel[local])}x{len(currentLevel[local][0])}"
    return quantidadeClasses, quantidadeImagens, resolucaoImagens      
quantidadeClasses, quantidadeImagens, resolucaoImagens = loadImages("./Banco de Imagens/", quantidadeClasses, quantidadeImagens, resolucaoImagens, bancoImagens)

## Tabela de Dados

Código adaptado do EP1.

In [9]:
def imprimeTabelaDetalhadaDeDados():
    print("\n=============================================================================\n")
    print("Tabela Detalhada de Dados:\n")
    dados = pandas.DataFrame({"Classe": [],
                         "N°Objetos": [], "N°Fundos": [],
                         "N°Iluminações": [], "Total": []})
    classes = bancoImagens.keys()
    for classe in classes:
        totalAmostras = 0
        totalFundos = []
        totalTempos = []
        totalLocais = []
        tempos = bancoImagens[classe].keys()
        for tempo in tempos:
            if tempo not in totalTempos: totalTempos.append(tempo)
            for local in bancoImagens[classe][tempo].keys():
                if local not in totalLocais: totalLocais.append(local)
                for fundo in bancoImagens[classe][tempo][local].keys():
                    if fundo not in totalFundos: totalFundos.append(fundo)
                    
        contadorFundos = len(totalFundos)
        contadorIluminacoes = len(totalTempos)*len(totalLocais)
        totalAmostras += contadorIluminacoes*contadorFundos
                    
        dados = pandas.concat([dados, pandas.DataFrame({"Classe": classe, "N°Objetos": 1, "N°Fundos": contadorFundos,
        "N°Iluminações": contadorIluminacoes, "Total": totalAmostras}, index=[0])], ignore_index = True)
    print(dados)
    print("\n===========================================================================\n")
imprimeTabelaDetalhadaDeDados()



Tabela Detalhada de Dados:

     Classe  N°Objetos  N°Fundos  N°Iluminações  Total
0   Alicate        1.0       6.0            4.0   24.0
1    Caneta        1.0       6.0            4.0   24.0
2    Colher        1.0       6.0            4.0   24.0
3  Controle        1.0       6.0            4.0   24.0
4   Esmalte        1.0       6.0            4.0   24.0
5      Faca        1.0       6.0            4.0   24.0
6     Garfo        1.0       6.0            4.0   24.0
7      Lixa        1.0       6.0            4.0   24.0
8     Moeda        1.0       6.0            4.0   24.0
9     Prato        1.0       6.0            4.0   24.0




## Save Image

Código com base na sugestão do enunciado.

In [11]:
def saveImage(filename, image):
    if not imwrite(filename, image):
        print("Não salvou imagem!")

## Conversão para níveis de cinza

Código baseado na sugestão do enunciado, irá converter imagens com 3 canais de cores para imagens em níveis de intensidade de cinza.
Também será armazenado em um dicionário o novo dataset com as imagens em níveis de cinza.

In [13]:
def RGB2Gray():
    if not os.path.exists("./originalGrayDataset/"):
        os.makedirs("./originalGrayDataset/")
    for classe in bancoImagens.keys():
        for tempo in bancoImagens[classe].keys():
            for iluminacao in bancoImagens[classe][tempo].keys():
                for fundo in bancoImagens[classe][tempo][iluminacao].keys():
                        imagemConvertida = cv2.cvtColor(bancoImagens[classe][tempo][iluminacao][fundo], cv2.COLOR_BGR2GRAY)
                        saveImage(f"./originalGrayDataset/{classe},{tempo},{iluminacao},{fundo}.jpg", imagemConvertida)
RGB2Gray()

In [14]:
bancoImagensCinza = {}
quantidadeClassesCinzas, quantidadeImagensCinzas, resolucaoImagensCinzas = loadImages("./originalGrayDataset/", quantidadeClasses, quantidadeImagens, resolucaoImagens, bancoImagensCinza)

# Data Augmentation

As funções abaixo foram baseadas nas criadas para execução das transformações da Lista 1

## Constrast Strecthing

In [18]:
def contrast_stretching(originalImage):
    image = originalImage.copy()
    image = 255 * (image - image.min()) / (image.max() - image.min())
    return image

## Log Transform
O parâmetro c desta função é adquirido com base no maior nível de intensidade presente na imagem, portanto a função atua de maneira dinâmica para as diferentes iluminações e condições do dataset.

In [20]:
def log_transform(originalImage):
    image = originalImage.copy()
    c = 255/np.log(np.max(np.array(image)))
    for lineIndex in range(len(image)):
        for columnIndex in range(len(image[lineIndex])):
            image[lineIndex][columnIndex] = c*(np.log(image[lineIndex][columnIndex]+1))
    return image

## Power Law Transform

In [22]:
def power_law_transform(originalImage, y, c):
    image = originalImage.copy()
    for lineIndex in range(len(image)):
        for columnIndex in range(len(image[lineIndex])):
            image[lineIndex][columnIndex] = c*(np.power(image[lineIndex][columnIndex],y))
    image = 255 * (image - image.min()) / (image.max() - image.min())
    return image

## Convolução 2D com Kernel para imagens em níveis de cinza

Função que realiza o processo de convolução entre um kernel de determinado filtro e uma imagem em níveis de cinza.
Primeiro criamos uma imagem com o tamanho da original com todos os valores zerados, e em seguida realizamos a convolução nas colunas, e depois, pegamos a imagem resultante e aplicamos a convolução nas linhas, obtendo assim a convolução de uma imagem em níveis de cinza com um kernel.

In [24]:
def convolucao2d(originalImage, kernel):

    image = np.zeros_like(originalImage)

    for i in range(len(originalImage)):
        image[i, :] = np.convolve(originalImage[i, :], kernel.ravel(), mode='same')

    for j in range(len(originalImage[0])):
        image[:, j] = np.convolve(image[:, j], kernel.ravel(), mode='same')
    
    return image

## Laplacian Image

Adquirimos o Laplaciano da imagem com base no kernel apresentado no livro Digital Image Processing, 4th Edition de Rafael C. Gonzalez e Richard E. Woods.
Utilizando esse kernel, podemos aplicar a convolução na imagem com este, para obter o Laplaciano da imagem original, este que pode ser utilizado posteriormente para servir como um filtro de sharpening da imagem.

In [26]:
def laplace_transform(originalImage):
    laplaceKernel= np.array([
        [ 0,  1,  0],
        [ 1, -4,  1],
        [ 0,  1,  0],
    ])
    
    return convolucao2d(originalImage, laplaceKernel)

## Mean Filter With Convolution

Realizamos a filtragem da imagem com o filtro da média, com base no kernel apresentado no livro Digital Image Processing, 4th Edition de Rafael C. Gonzalez e Richard E. Woods.
Utilizando esse kernel, podemos aplicar a convolução na imagem com este, para obter o a imagem filtrada, processo que pelo filtro de média é caracterizado como uma passagem low pass que resulta em suavização da imagem.

In [28]:
def mean_filter_convolution(originalImage):
    meanKernel = np.array([
        [ 1/9, 1/9, 1/9],
        [ 1/9, 1/9, 1/9],
        [ 1/9, 1/9, 1/9],
    ])
    
    return convolucao2d(originalImage, meanKernel)

## Aplicação

Aqui será realizada a aplicação de todos os métodos de Data Augmentation discutidos anteriormente em cada imagem do originalGrayDataset.
Com isso, teremos um novo dataset denominado augmentedDataset, que terá o resultado de todas as imagens processadas por cada um dos processos aqui descritos.

Para o caso especial da aplicação da Power Law transform, temos que valoresde gamma acima de 1 resultam em uma imagem mais escurecida com contraste maior, e valores abaixo de 1 resultam em imagens com contraste menor e mais claras, portanto, o valor de gamma foi controlado para que em casos de iluminação mais clara tenha valores de gamma acima de 1, e imagens com iluminação mais escura tenha valores de gamma abaixo de 1.

In [30]:
def data_augmentation():
    gamma = 1
    
    if not os.path.exists("./augmentedDataset/"):
        os.makedirs("./augmentedDataset/")

    for classe in bancoImagensCinza.keys():
        for tempo in bancoImagensCinza[classe].keys():
            for iluminacao in bancoImagensCinza[classe][tempo].keys():
                for fundo in bancoImagensCinza[classe][tempo][iluminacao].keys():
                    imagemConstrastStretching = contrast_stretching(bancoImagensCinza[classe][tempo][iluminacao][fundo])
                    if(tempo == "Dia"):
                        if(iluminacao == "Dentro"): gamma = 2
                        else: gamma = 1.5
                    else:
                        if(iluminacao == "Dentro"): gamma = 0.9
                        else: gamma = 0.7
                    imagemLog = log_transform(bancoImagensCinza[classe][tempo][iluminacao][fundo])
                    imagemPowerLaw = power_law_transform(bancoImagensCinza[classe][tempo][iluminacao][fundo], gamma, 1)
                    imagemLaplaceTransformed = laplace_transform(bancoImagensCinza[classe][tempo][iluminacao][fundo])
                    imagemMeanFilterConvolution = mean_filter_convolution(bancoImagensCinza[classe][tempo][iluminacao][fundo])
                        
                    saveImage(f"./augmentedDataset/{classe},{tempo},{iluminacao},{fundo},ContrastStrechted.jpg", imagemConstrastStretching)
                    saveImage(f"./augmentedDataset/{classe},{tempo},{iluminacao},{fundo},Log.jpg", imagemLog)
                    saveImage(f"./augmentedDataset/{classe},{tempo},{iluminacao},{fundo},PowerLaw.jpg", imagemPowerLaw)
                    saveImage(f"./augmentedDataset/{classe},{tempo},{iluminacao},{fundo},Laplace.jpg", imagemLaplaceTransformed)
                    saveImage(f"./augmentedDataset/{classe},{tempo},{iluminacao},{fundo},MeanFilterConvolution.jpg", imagemMeanFilterConvolution)
data_augmentation()

KeyboardInterrupt: 

## Visualização

Funções de visualização dos datasets com base nas desenvolvidas no EP1.

In [ ]:
#Código adaptado da Lista 1
def plot_img(fig, titulo):
  plt.figure(figsize=(3,3))
  plt.imshow(fig, cmap='gray', vmin=0, vmax=255)
  plt.title(titulo)
  plt.axis('off')
  plt.show()
    
def imprimeImagensPorClasse(bancoImagens):
    for classe in bancoImagens.keys():
        for tempo in bancoImagens[classe].keys():
            for iluminacao in bancoImagens[classe][tempo].keys():
                for fundo in bancoImagens[classe][tempo][iluminacao].keys():
                    plot_img(bancoImagens[classe][tempo][iluminacao][fundo], f"Classe: {classe}\nTempo: {tempo}\nLocal: {iluminacao}\nFundo: {fundo}")

## OriginalGrayDataset

Realiza a visualização do originalGrayDataset

In [ ]:
imprimeImagensPorClasse(bancoImagensCinza)

## AugmentedDataset

Armazena o augmentedDataset em um dicionário e o visualiza.

In [ ]:
def imprimeImagensProcessadasPorClasse(bancoImagens):
    for classe in bancoImagens.keys():
        for tempo in bancoImagens[classe].keys():
            for iluminacao in bancoImagens[classe][tempo].keys():
                for fundo in bancoImagens[classe][tempo][iluminacao].keys():
                    for processo in bancoImagens[classe][tempo][iluminacao][fundo].keys():
                        print(tempo)
                        print(iluminacao)
                        plot_img(bancoImagens[classe][tempo][iluminacao][fundo][processo], f"Classe: {classe}\nTempo: {tempo}\nLocal: {iluminacao}\nFundo: {fundo}\nProcesso: {processo}")
bancoImagensProcessadas = {}
quantidadeClasses, quantidadeImagens, resolucaoImagens = loadImages("./augmentedDataset/", quantidadeClasses, quantidadeImagens, resolucaoImagens, bancoImagensProcessadas)
imprimeImagensProcessadasPorClasse(bancoImagensProcessadas)